# The survey
***

<img align="right" src="https://github.com/3dgeo-heidelberg/helios/blob/dev/h++.png?raw=true" width="300">

This page will give an introduction on using `pyhelios` to access and modify surveys.

`pyhelios` allows to:

- obtain scanning device characteristics
- calculate the length of a survey
- view and modify the scanner and platform settings at each leg

In [ ]:
import sys
import os
from pathlib import Path
import math

current_folder = globals()["_dh"][0]
helios_path = str(Path(current_folder).parent)
sys.path.append(helios_path)  # add helios-plusplus directory to PATH
import pyhelios

os.chdir(helios_path)

In [ ]:
pyhelios.loggingDefault()
# build simulation parameters
simBuilder = pyhelios.SimulationBuilder(
    "data/surveys/toyblocks/als_toyblocks.xml", "assets/", "output/"
)
simBuilder.setNumThreads(0)
simBuilder.setLasOutput(True)
simBuilder.setZipOutput(True)

# build the survey
simB = simBuilder.build()

Once we built a survey, we have numerous options to obtain and change the characteristics of all components of our simulation. Note that after the steps above, simB is a SimulationBuild object. To access the Simulation itself, we have to call simB.sim.

In [ ]:
# obtain survey path and name
survey_path = simB.sim.getSurveyPath()
survey = simB.sim.getSurvey()
survey_name = survey.name
print(survey_name)

We can also obtain the survey length, i.e. the distance through all waypoints.
If the survey has not been running yet, `survey.getLength()` will return 0.0.
We can calculate the length of a loaded survey of a simulation which was built but not started with `survey.calculateLength()`.


In [ ]:
survey.getLength()

In [ ]:
survey.calculateLength()
print(survey.getLength())

## The scanner

Let's have a look at the scanner we are using.

In [ ]:
scanner = simB.sim.getScanner()
# print scanner characteristics
print(scanner.toString())

The scanner characteristics can also be accessed individually:

In [ ]:
print(
    f"""
Device ID: \t\t{scanner.deviceId}
Average power: \t{scanner.averagePower} W
Beam divergence: \t{scanner.beamDivergence} rad
Wavelength: \t{scanner.wavelength*1000000000} nm
Scanner visibility \t{scanner.visibility} m
"""
)

The scanner has also some more properties:

In [ ]:
print(
    f"""
Number of subsampling rays: \t\t{scanner.numRays}
Pulse length: \t\t\t{scanner.pulseLength_ns} ns
Supported pulse frequenceis: \t{list(scanner.getSupportedPulseFrequencies())} Hz
"""
)

We can also get information about the scanner head, e.g. the maximum rotation speed in case of TLS scanners.
Let's load a TLS survey.

In [ ]:
pyhelios.loggingDefault()
# build simulation parameters
simBuilder = pyhelios.SimulationBuilder(
    "data/surveys/demo/tls_arbaro_demo.xml", "assets/", "output/"
)
simBuilder.setNumThreads(0)
simBuilder.setLasOutput(True)
simBuilder.setZipOutput(True)

# build the survey
simB = simBuilder.build()

In [ ]:
scanner = simB.sim.getScanner()
print(scanner.toString())

In [ ]:
head = scanner.getScannerHead()
# get scanner rotation speed and range
print(
    f"""
Max. rotation speed: {round(head.rotatePerSecMax * 180 / math.pi)} degrees per second
"""
)

If we want to obtain the scanning mechanism, we have to get the beam deflector.

In [ ]:
deflector = scanner.getBeamDeflector()
print(
    f"""
Scan frequency range: {deflector.scanFreqMin} - {deflector.scanFreqMax} Hz
Scan angle range: {round(deflector.scanAngleMax * 180 / math.pi)}° FOV
"""
)
# deflector.optics

# not needed:
# deflector.verticalAngleMin
# deflector.verticalAngleMax

From the beam detector, we get information about, e.g., the accuracy of the scanner.

In [ ]:
detector = scanner.getDetector()
print(
    f"""
Accuracy: {detector.accuracy} m
Minimum range: {detector.rangeMin} m
Maximum range: {detector.rangeMax} m
"""
)
# detector().maxNOR

# crashes when executing:
# detector.lasScale

We can also get the scanner full waveform settings.
Like many of the scanner settings, they can be overwritten in the `scannerSettings` of a leg.

In [ ]:
print(
    f"""Full waveform settings for {scanner.deviceId}
Bin size: {scanner.fwfSettings.binSize_ns} ns
Window size: {scanner.fwfSettings.winSize_ns} ns
Beam sample quality: {scanner.fwfSettings.beamSampleQuality}
"""
)

## Legs

Each leg of a survey has scanner settings and platform settings, (cf. survey XML file),
which can be accessed and changed with pyhelios.


In [ ]:
# get the first leg
leg = simB.sim.getLeg(0)

# scanner settings
print(
    f"""
Scanner is active: \t{leg.getScannerSettings().active}
Pulse frequency: \t{leg.getScannerSettings().pulseFreq} Hz
Scan angle: \t\t{leg.getScannerSettings().scanAngle * 180 / math.pi}°
Minimum vertical angle: {leg.getScannerSettings().verticalAngleMin * 180 / math.pi}°
Maximum vertical angle: {round(leg.getScannerSettings().verticalAngleMax * 180 / math.pi)}°
Scan frequency: \t{leg.getScannerSettings().scanFreq} Hz
Beam divergence: \t{leg.getScannerSettings().beamDivAngle * 1000} mrad
Trajectory time interval: \t{leg.getScannerSettings().trajectoryTimeInterval} s
Start angle of head rotation: \t{leg.getScannerSettings().headRotateStart * 180 / math.pi}°
Start angle of head rotation: \t{leg.getScannerSettings().headRotateStop * 180 / math.pi}°
Rotation speed: \t\t{leg.getScannerSettings().headRotatePerSec * 180 / math.pi}° per s
"""
)

Scanner Settings and platform settings may be defined through a template.
For this, let's first switch back to the ALS toyblocks demo

In [ ]:
simBuilder = pyhelios.SimulationBuilder(
    "data/surveys/toyblocks/als_toyblocks.xml", "assets/", "output/"
)
simBuilder.setNumThreads(0)
simBuilder.setLasOutput(True)
simBuilder.setZipOutput(True)
simB = simBuilder.build()

The template can be accessed for a given ScannerSettings or PlatformSettings instance:

In [ ]:
leg = simB.sim.getLeg(0)

ss = leg.getScannerSettings()
if ss.hasTemplate():
    ss_tmpl = ss.getTemplate()
    print(
        f"""
    Template name: {ss_tmpl.id}
    Pulse frequency: {ss_tmpl.pulseFreq/1000} kHz
    """
    )  # Print the pulse frequency defined in the template

In [ ]:
ps = leg.getPlatformSettings()
if ps.hasTemplate():
    ps_tmpl = ps.getTemplate()
    print(
        f"""
    Platform template name: {ps_tmpl.id}
    Speed: {ps_tmpl.movePerSec} m/s
    Altitude: {ps_tmpl.z} m
    """
    )

We can also change the template.

In [ ]:
ps_tmpl.z += 20
print(f"New altitude: {ps_tmpl.z} m")

Some further platform settings:

In [ ]:
print(
    f"""
On ground? {leg.getPlatformSettings().onGround}
Position: ({leg.getPlatformSettings().x}, {leg.getPlatformSettings().y}, {leg.getPlatformSettings().z})
"""
)

If we compare the position here to the position in the XML survey file, we notice that they do not match.
The difference is 50 in x direction and 70 in y direction.

When loading a survey, **a shift is applied to the scene and to each leg**. We can obtain this shift:

In [ ]:
scene = simB.sim.getScene()
shift = scene.getShift()
print(f"Shift = ({shift.x},{shift.y},{shift.z})")

Using a for-loop, we can get all leg positions.
Note that we add the shift to obtain the true coordinates as specified in the XML-file:


In [ ]:
for i in range(simB.sim.getNumLegs()):
    leg = simB.sim.getLeg(i)
    print(
        f"Leg {i}\tposition = "
        f"{leg.getPlatformSettings().x+shift.x},"
        f"{leg.getPlatformSettings().y+shift.y},"
        f"{leg.getPlatformSettings().z+shift.z}\t"
        f"active = {leg.getScannerSettings().active}"
    )

We can also use a for-loop to create new legs.
Here an example, where we initiate a simulation with a survey with no legs (`data/surveys/default_survey.xml`) and
then create the legs with Python.

In [ ]:
pyhelios.loggingDefault()
default_survey_path = "data/surveys/default_survey.xml"

# default survey with the toyblocks scene (missing platform and scanner definition and not containing any legs)
survey = """
<?xml version="1.0" encoding="UTF-8"?>
<document>
    <survey name="some_survey" scene="data/scenes/toyblocks/toyblocks_scene.xml#toyblocks_scene" platform="data/platforms.xml#copter_linearpath" scanner="data/scanners_als.xml#riegl_vux-1uav">
    </survey>
</document>
"""

with open(default_survey_path, "w") as f:
    f.write(survey)

simBuilder = pyhelios.SimulationBuilder(default_survey_path, "assets/", "output/")
simBuilder.setCallbackFrequency(10)
simBuilder.setLasOutput(True)
simBuilder.setZipOutput(True)
simBuilder.setRebuildScene(True)

simB = simBuilder.build()

waypoints = [
    [100.0, -100.0],
    [-100.0, -100.0],
    [-100.0, -50.0],
    [100.0, -50.0],
    [100.0, 0.0],
    [-100.0, 0.0],
    [-100.0, 50.0],
    [100.0, 50.0],
    [100.0, 100.0],
    [-100.0, 100.0],
]
altitude = 100
speed = 150
pulse_freq = 300_000
scan_freq = 200
scan_angle = 37.5 / 180 * math.pi  # convert to rad
shift = simB.sim.getScene().getShift()
for j, wp in enumerate(waypoints):
    leg = simB.sim.newLeg(j)
    leg.serialId = j  # assigning a serialId is important!
    leg.getPlatformSettings().x = wp[0] - shift.x  # don't forget to apply the shift!
    leg.getPlatformSettings().y = wp[1] - shift.y
    leg.getPlatformSettings().z = altitude - shift.z
    leg.getPlatformSettings().movePerSec = speed
    leg.getScannerSettings().pulseFreq = pulse_freq
    leg.getScannerSettings().scanFreq = scan_freq
    leg.getScannerSettings().scanAngle = scan_angle
    leg.getScannerSettings().trajectoryTimeInterval = (
        0.05  # important to get a trajectory output
    )
    if j % 2 != 0:
        leg.getScannerSettings().active = False

Let's execute this survey!

In [ ]:
import time

start_time = time.time()
simB.start()

if simB.isStarted():
    print("Simulation is started!")

while simB.isRunning():
    duration = time.time() - start_time
    mins = duration // 60
    secs = duration % 60
    print(
        "\r"
        + "Simulation is running since {} min and {} sec. Please wait.".format(
            int(mins), int(secs)
        ),
        end="",
    )
    time.sleep(1)

output = simB.join()
print("\nSimulation has finished.")

Now let us also quickly visualize the output. We load the points into numpy arrays using the function `outputToNumpy` and then visualize the point cloud with matplotlib as a simple top view with points coloured by the `hitObjectId`.

In [ ]:
import matplotlib.pyplot as plt

pc, trajectory = pyhelios.outputToNumpy(output)

# Matplotlib figure.
fig = plt.figure(figsize=(5, 5))
# Axes3d axis onto mpl figure.
ax = fig.add_subplot()

# Scatter plot of original and simulated points in different colors
ax.scatter(pc[:, 0], pc[:, 1], c=pc[:, 14], s=0.01)

# Add axis labels.
ax.set_xlabel("$X$")
ax.set_ylabel("$Y$")
ax.axis("equal")

# Set title.
from textwrap import wrap

title = ax.set_title(
    "\n".join(
        wrap(
            "Top view of the simulated point cloud, coloured by " + r"$hitObjectId$", 40
        )
    )
)

plt.show()